In [50]:
# required libraries
import csv
import json
import pandas as pd
import datetime as dt
import sqlite3
from sqlalchemy import create_engine
import geocoder
from itertools import islice
import time

In [2]:
# convert CSV data to DataFrame
csvfile = open('./data/restaurant-and-market-health-violations.csv', 'r')
pdCSV = pd.DataFrame.from_csv(csvfile,index_col=24)
pdCSV.head()
csvfile.close()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
# filter data for the July 2018 only
pdDate = pdCSV.loc[(pdCSV['activity_date'] > '2018-06-30')&(pdCSV['activity_date'] <='2018-07-31')]

In [70]:
# save filtered CSV data as JSON file
pdDate.to_json(r'./data/restaurant-and-market-health-violations.json', orient='records')
pdDate.columns

Index(['serial_number', 'activity_date', 'facility_name', 'violation_code',
       'violation_description', 'violation_status', 'points', 'grade',
       'facility_address', 'facility_city', 'facility_id', 'facility_state',
       'facility_zip', 'employee_id', 'owner_id', 'owner_name',
       'pe_description', 'program_element_pe', 'program_name',
       'program_status', 'record_id', 'score', 'service_code',
       'service_description'],
      dtype='object')

In [12]:
# save filtered CSV data as CSV file and Export DataFrame to SQLLITE
pdDate.to_csv(r'./data/restaurant-and-market-health-violations-0718.csv')
engine = create_engine('sqlite:///data/los_angeles.db', echo=False)
pdDate.to_sql('losangeles', con=engine, if_exists='append')

In [36]:
# generate DataFrame to contain location data
dfLoc = pdDate[['facility_id','facility_address','facility_city','facility_state','facility_zip']].drop_duplicates()
dfLoc.set_index('facility_id')
dfLoc.head()

,facility_id,facility_address,facility_city,facility_state,facility_zip
row_id,,,,,
DA01KEPWCF045,FA0179718,346 N FORD BLVD,LOS ANGELES,CA,90022-1121
DA006DSLSF006,FA0251976,541 S SPRING ST # 114,LOS ANGELES,CA,90013
DA00GQVTQF008,FA0240579,4030 S CENTINELA AVE,LOS ANGELES,CA,90066
DA00RZ5GNF008,FA0165233,13350 WASHINGTON BLVD,LOS ANGELES,CA,90066
DA01T9PYHF032,FA0222724,3109 N SAN FERNANDO RD,LOS ANGELES,CA,90065


In [65]:
# geocode location data
LIMIT = 1800
count = 0
for index, row in  islice(dfLoc.iterrows(), LIMIT):
    addr = row['facility_address'].replace("1/2","").split("#")
    location = addr[0].replace("1/4","")+ ", "+row['facility_city']+ ", "+row['facility_state']+ " "+row['facility_zip']
    print(location)
    g = geocoder.mapbox(location,key= "pk.eyJ1Ijoiam9yZ2VwaXJlcyIsImEiOiJjanZvajBudGMwYmptNDRxbG95cWU0ZW0yIn0.x51Y6jI0l8i17yi1qdANpA")
    print(f'{g.lat} {g.lng}') 
    dfLoc.loc[index,'lat'] = g.lat
    dfLoc.loc[index,'lng'] = g.lng
    if(count == 599):
        time.sleep(20)

346 N FORD BLVD, LOS ANGELES, CA 90022-1121
34.032087 -118.168932
541 S SPRING ST , LOS ANGELES, CA 90013
34.04657 -118.2507
4030 S CENTINELA AVE, LOS ANGELES, CA 90066
33.998875 -118.429599
13350 WASHINGTON BLVD, LOS ANGELES, CA 90066
33.992318 -118.443552
3109 N SAN FERNANDO RD, LOS ANGELES, CA 90065
34.115162 -118.245892
5621 N FIGUEROA ST, LOS ANGELES, CA 90042
34.10938 -118.1936
700 W CESAR E CHAVEZ AVE, LOS ANGELES, CA 90012
34.06081 -118.244369
7075 W SUNSET BLVD, LOS ANGELES, CA 90028
34.09813 -118.34386
9616 W PICO BLVD, LOS ANGELES, CA 90035
34.05497 -118.39811
2500 S CENTRAL AVE, LOS ANGELES, CA 90011
34.01966 -118.25438
9860 NATIONAL BLVD, LOS ANGELES, CA 90034
34.030708 -118.401697
1320 E 41ST ST, LOS ANGELES, CA 90011
34.00903 -118.25132
3400 W 8TH ST, LOS ANGELES, CA 90005
34.05755 -118.30334
1457 E FLORENCE AVE STE , LOS ANGELES, CA 90001
33.97506 -118.24816
524  N LA BREA AVE, LOS ANGELES, CA 90036
34.08103 -118.34385
419 N FAIRFAX AVE, LOS ANGELES, CA 90036
34.07859 -

34.08369 -118.18867
840 S SPRING ST, LOS ANGELES, CA 90014
34.04233 -118.25416
8448 LINCOLN BLVD, LOS ANGELES, CA 90045
33.960965 -118.420013
3250 W OLYMPIC BLVD STE 311, LOS ANGELES, CA 90006
34.05244 -118.30766
4501 MELROSE AVE, LOS ANGELES, CA 90029
34.083728 -118.298171
10628 S SAN PEDRO ST, LOS ANGELES, CA 90003-4934
33.93942 -118.26934
1300 W OLYMPIC BLVD , LOS ANGELES, CA 90015
34.04728 -118.27036
510 W 7TH ST , LOS ANGELES, CA 90014-2502
34.04671 -118.25598
7307 N FIGUEROA ST , LOS ANGELES, CA 90041
34.13532 -118.18811
2112 BEVERLY BLVD, LOS ANGELES, CA 90057
34.06664 -118.27044
700 CYPRESS AVE, LOS ANGELES, CA 90065
34.09093 -118.2237
928 S WESTERN AVE STE 125, LOS ANGELES, CA 90006
34.05485 -118.30886
2844 WABASH AVE, LOS ANGELES, CA 90033
34.049734 -118.196722
5640 W ADAMS BLVD, LOS ANGELES, CA 90016
34.03225 -118.3663
1820 MARENGO ST, LOS ANGELES, CA 90033
34.05809 -118.21128
801 S VERMONT AVE , LOS ANGELES, CA 90005
34.05755 -118.29182
4114 SANTA MONICA BLVD, LOS ANGELES, 

34.04768 -118.43513
100 WORLD WAY , LOS ANGELES, CA 90045
33.94572 -118.40095
5482 E BEVERLY BLVD, LOS ANGELES, CA 90022
34.0295 -118.1495
1606 BEVERLY BLVD, LOS ANGELES, CA 90026
34.06351 -118.26351
8326 S WESTERN AVE, LOS ANGELES, CA 90047
33.96354 -118.30891
7224 S VERMONT AVE, LOS ANGELES, CA 90044
33.97406 -118.2913
7353 MELROSE AVE, LOS ANGELES, CA 90046
34.08364 -118.34988
263 S ALEXANDRIA AVE, LOS ANGELES, CA 90004
34.06933 -118.29805
4050 MARLTON AVE, LOS ANGELES, CA 90008
34.01133 -118.33847
1449 WESTWOOD BLVD, LOS ANGELES, CA 90064
34.0548 -118.44141
6333 W 3RD ST , LOS ANGELES, CA 90036
34.0722 -118.36037
1156 N HIGHLAND AVE, LOS ANGELES, CA 90038-1205
34.0924 -118.33839
5212 W ADAMS BLVD, LOS ANGELES, CA 90016
34.03196 -118.35601
512 E WASHINGTON BLVD, LOS ANGELES, CA 90015
34.02812 -118.26012
10227 VENICE BLVD, LOS ANGELES, CA 90034
34.02153 -118.40273
4339  W SUNSET BLVD, LOS ANGELES, CA 90029
34.09499 -118.2827
8875 W PICO BLVD, LOS ANGELES, CA 90035
34.05535 -118.38519

34.01728 -118.25536
3525 W 3RD ST , LOS ANGELES, CA 90020
34.0693 -118.29253
811 WILSHIRE BLVD STE 2100, LOS ANGELES, CA 90017
34.049729 -118.2584
12105 W PICO BLVD, LOS ANGELES, CA 90064
34.02939 -118.4506
1802 N VERMONT AVE, LOS ANGELES, CA 90027
34.10378 -118.29162
11151 AVALON BLVD 110, LOS ANGELES, CA 90061
33.93392 -118.26524
5399 W CENTINELA AVE, LOS ANGELES, CA 90045
33.977458 -118.373793
10429 VENICE BLVD, LOS ANGELES, CA 90034
34.02032 -118.4049
5656 SANTA MONICA BLVD E, LOS ANGELES, CA 90038
34.090585 -118.313151
429 N WESTERN AVE , LOS ANGELES, CA 90004-2600
34.07894 -118.30933
1636 WESTWOOD BLVD, LOS ANGELES, CA 90024
34.052321 -118.438951
1430 S FAIRFAX AVE, LOS ANGELES, CA 90019
34.05038 -118.36661
1383 WESTWOOD BLVD, LOS ANGELES, CA 90024
34.0555 -118.44211
5615 HOLLYWOOD BLVD, LOS ANGELES, CA 90027
34.1018 -118.312
2020 HOOPER AVE, LOS ANGELES, CA 90011
34.01981 -118.24711
454 W FLORENCE AVE STE 104, LOS ANGELES, CA 90003
33.97457 -118.2819
12009 WILSHIRE BLVD, LOS ANG

34.0482 -118.46241
5185 W SUNSET BLVD, LOS ANGELES, CA 90027
34.09828 -118.30305
4541 EAGLE ROCK BLVD, LOS ANGELES, CA 90041
34.1271 -118.218585
148 S BARRINGTON AVE, LOS ANGELES, CA 90049
34.06475 -118.46937
4166 S BROADWAY, LOS ANGELES, CA 90037
34.00759 -118.27813
107 E 6TH ST, LOS ANGELES, CA 90013
34.04522 -118.24969
3950 W MLK BLVD, LOS ANGELES, CA 90008
34.01499 -118.34203
698 S VERMONT AVE , LOS ANGELES, CA 90005
34.05979 -118.29154
125 N LARCHMONT BLVD, LOS ANGELES, CA 90004
34.07382 -118.32383
750 N HILL ST STE F, LOS ANGELES, CA 90012-2383
34.06203 -118.2397
10948 WEYBURN AVE, LOS ANGELES, CA 90024
34.06239 -118.44669
5633 HOLLYWOOD BLVD, LOS ANGELES, CA 90028
34.10203 -118.31294
11301 W OLYMPIC BLVD STE 204, LOS ANGELES, CA 90064
34.03819 -118.44181
1631 W SUNSET BLVD, LOS ANGELES, CA 90026
34.07733 -118.25815
8722 W 3RD ST, LOS ANGELES, CA 90046
34.07349 -118.38236
405 N ALVARADO ST, LOS ANGELES, CA 90026
34.07065 -118.26798
1753 E FLORENCE AVE, LOS ANGELES, CA 90001
33.97

...

34.03953 -118.26053
4433 S ALAMEDA ST , LOS ANGELES, CA 90058
34.00295 -118.23924
831 S VERMONT AVE, LOS ANGELES, CA 90006
34.05662 -118.29171
841 S VERMONT AVE, LOS ANGELES, CA 90005
34.05662 -118.29171
11913 W OLYMPIC BLVD, LOS ANGELES, CA 90064
34.03279 -118.45272
2619 W SUNSET BLVD, LOS ANGELES, CA 90026
34.0804 -118.26999
3559 E 1ST ST, LOS ANGELES, CA 90063
34.0363 -118.19053
450 S WESTERN AVE, LOS ANGELES, CA 90020
34.06562 -118.30887
4711 S BROADWAY, LOS ANGELES, CA 90037
34.00081 -118.27841
11870 SANTA MONICA BLVD STE 101, LOS ANGELES, CA 90025
34.0419 -118.45979
4504 S AVALON BLVD, LOS ANGELES, CA 90011
34.00228 -118.26504
816 W 8TH ST, LOS ANGELES, CA 90014
34.04725 -118.26026
6333 W 3RD ST , LOS ANGELES, CA 90036
34.0722 -118.36037
12800 S AVALON BLVD, LOS ANGELES, CA 90061
33.916211 -118.264929
2174 E 103RD ST, LOS ANGELES, CA 90002
33.94301 -118.23296
123 N LARCHMONT BLVD, LOS ANGELES, CA 90004
34.07377 -118.32382
620 S HARVARD BLVD, LOS ANGELES, CA 90005
34.06286 -118.30

34.05542 -118.39261
11740 SAN VINCENTE BLVD , LOS ANGELES, CA 90049
34.053083 -118.466904
1114 N ST ANDREWS PL, LOS ANGELES, CA 90038
34.09122 -118.31077
3400 E 8TH ST, LOS ANGELES, CA 90023
34.02132 -118.20468
5833 FRANKLIN AVE, LOS ANGELES, CA 90028
34.10535 -118.31644
1202 ABBOT KINNEY BLVD, LOS ANGELES, CA 90291
33.991279 -118.469023
6335 W WILSHIRE BLVD, LOS ANGELES, CA 90048
34.063746 -118.36648
500 WORLD WAY, LOS ANGELES, CA 90045
33.94251 -118.405071
119 E 7TH ST, LOS ANGELES, CA 90014
34.04361 -118.25061
5325 S FIGUEROA ST, LOS ANGELES, CA 90037
33.99347 -118.28287
2180 WESTWOOD BLVD , LOS ANGELES, CA 90025
34.04421 -118.431802
2214 W SUNSET BLVD, LOS ANGELES, CA 90026
34.07815 -118.26664
455 W FLORENCE AVE, LOS ANGELES, CA 90003
33.97483 -118.28217
3535 E 1ST ST, LOS ANGELES, CA 90063
34.036185 -118.191169
345 N LA BREA AVE , LOS ANGELES, CA 90036
34.07755 -118.34424
10901 S MAIN ST, LOS ANGELES, CA 90061
33.93686 -118.2742
4034 W MARTIN LUTHER KING JR BLVD, LOS ANGELES, CA 9

33.995112 -118.437824
12630  WASHINGTON PL, LOS ANGELES, CA 90066
33.997116 -118.434436
3029 W 8TH ST, LOS ANGELES, CA 90005
34.0579 -118.29239
818 WILSHIRE BLVD STE , LOS ANGELES, CA 90017
34.049666 -118.258912
2745 BEVERLY BLVD, LOS ANGELES, CA 90057
34.070646 -118.278984
1117 N HOBART BLVD, LOS ANGELES, CA 90029
34.09134 -118.30476
608 S NEW HAMPSHIRE AVE, LOS ANGELES, CA 90005
34.06327 -118.29275
450 S WESTERN AVE, LOS ANGELES, CA 90020
34.06562 -118.30887
610 S BERENDO ST, LOS ANGELES, CA 90005
34.06327 -118.29396
3113 N SAN FERNANDO RD, LOS ANGELES, CA 90065
34.115199 -118.24593
304 N WESTERN AVE, LOS ANGELES, CA 90004
34.0765 -118.30892
3771 E 1ST ST, LOS ANGELES, CA 90063
34.036218 -118.184751
1727 N VERMONT AVE STE 102, LOS ANGELES, CA 90027
34.10259 -118.29205
3827 W SUNSET BLVD STE C, LOS ANGELES, CA 90026
34.09153 -118.27923
101 E MANCHESTER AVE, LOS ANGELES, CA 90003
33.96038 -118.27285
4212 S CENTRAL AVE, LOS ANGELES, CA 90011
34.00716 -118.2561
3101 W 8TH ST, LOS ANGELES

34.00799 -118.27826
3900 WILSHIRE BLVD, LOS ANGELES, CA 90010
34.0616 -118.31133
923 E 3RD ST STE 101, LOS ANGELES, CA 90013
34.04597 -118.23457
4400 S BROADWAY, LOS ANGELES, CA 90037
34.00378 -118.27825
698 S IROLO ST STE , LOS ANGELES, CA 90005
34.05974 -118.3008
8424 BEVERLY BLVD, LOS ANGELES, CA 90048
34.07596 -118.37345
4531 W WASHINGTON BLVD, LOS ANGELES, CA 90016
34.03993 -118.33902
319 E 2ND ST STE , LOS ANGELES, CA 90012
34.04929 -118.24098
4319 W 2ND ST, LOS ANGELES, CA 90004
34.07123 -118.30814
849 S VERMONT AVE, LOS ANGELES, CA 90005
34.05662 -118.29171
1901 S ALAMEDA ST STE 115, LOS ANGELES, CA 90058-1037
34.01973 -118.23974
5808 RODEO RD, LOS ANGELES, CA 90016
34.02171 -118.37521
710 S HILL ST, LOS ANGELES, CA 90014
34.04549 -118.25471
5811 S COMPTON AVE, LOS ANGELES, CA 90011
33.9898 -118.24783
5259 HOLLYWOOD BLVD, LOS ANGELES, CA 90027
34.10191 -118.30548
3470 W 6TH ST , LOS ANGELES, CA 90020
34.06339 -118.29748
4600 MELROSE AVE, LOS ANGELES, CA 90029
34.08345 -118.3015

34.076969 -118.168671
300 S SANTA FE AVE, LOS ANGELES, CA 90013
34.04723 -118.24959
3915 W OLYMPIC BLVD, LOS ANGELES, CA 90019
34.05378 -118.32005
5039 WHITTIER BLVD, LOS ANGELES, CA 90022
34.021427 -118.160591
313 N FIGUEROA ST 10TH FLOOR, LOS ANGELES, CA 90012
34.05972 -118.24939
5300 S BROADWAY, LOS ANGELES, CA 90037
33.99402 -118.27821
11927 SANTA MONICA BLVD, LOS ANGELES, CA 90025
34.04172 -118.46087
998 S ROBERTSON BLVD, LOS ANGELES, CA 90035
34.05969 -118.38343
1224 S ATLANTIC BLVD, LOS ANGELES, CA 90022
34.015458 -118.158543
2201 S SAN PEDRO ST, LOS ANGELES, CA 90011
34.02483 -118.25946
7219 S ALAMEDA ST, LOS ANGELES, CA 90001
33.973865 -118.234806
6517 HOLLYWOOD BLVD, LOS ANGELES, CA 90028
34.10175 -118.33157
900 W OLYMPIC BLVD, LOS ANGELES, CA 90015
34.04526 -118.26611
709 N LA BREA AVE, LOS ANGELES, CA 90038
34.08384 -118.34424
377 S ATLANTIC BLVD, LOS ANGELES, CA 90022
34.030138 -118.154559
2506  E 1ST ST, LOS ANGELES, CA 90033
34.04287 -118.20839
1637 SILVER LAKE BLVD, LOS

In [67]:
dfLoc.count()

facility_id         1746
facility_address    1746
facility_city       1746
facility_state      1746
facility_zip        1746
lat                 1746
lng                 1746
dtype: int64

In [68]:
# post geocode to locations table
dfLoc.to_sql('locations', con=engine, if_exists='append')

In [73]:
# create scoring dataframe
dfScoring = pdDate[['facility_id','score','grade']]
dfScoring.head()
dfScoring.to_sql('scoring', con=engine, if_exists='append')